In [1]:
from pennylane.optimize import AdamOptimizer,QNSPSAOptimizer,AdagradOptimizer
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
import jax
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as mse
from scipy.optimize import minimize
from IPython.display import clear_output
import random 
from pennylane.optimize import AdamOptimizer,QNSPSAOptimizer
from utils import *
from autoencoder import Autoencoder
import os 
import pennylane as qml
import jax
import jax.numpy as jnp
from Cluster.autoencoder6 import JAxutoencoder


In [ ]:
def create_ising_state(self,sv,nq,start=0,):
    qml.QubitStateVector(sv, wires=range(nq+start))

def compare_state_orig(n_qb_input):
    @qml.qnode(qml.device('default.mixed', wires=n_qb_input*2+1, shots=1000))
    def pio(param):
        print(param[0])
        create_ising_state(param[0],n_qb_input,1)
        create_ising_state(param[1],n_qb_input,n_qb_input+1)
        
        qml.Barrier()
        original_swap([(1+a,1+a+n_qb_input) for a in range(n_qb_input)])
        return qml.probs([0])
    return pio

def compare_state_ae(n_qb_input,n_qb_trash,ae):
    @qml.qnode(qml.device('default.mixed', wires=n_qb_input*2+1, shots=1000))
    def pio(param):
        create_ising_state(param[0],n_qb_input,1)
        create_ising_state(param[1],n_qb_input,n_qb_input+1)
        
        qml.Barrier()
        ae.get_cirq(1)
        ae.get_cirq(n_qb_input+1)

        qml.Barrier()
        original_swap([(1+n_qb_trash+a,1+n_qb_trash+a+n_qb_input) for a in range(n_qb_input-n_qb_trash)])
        return qml.probs([0])
    return pio

def compare_matrix_fidelity(n_qubit_autoencoder,n_trash_qubit,ae,loc=9):
    m1=[]
    data=get_data(n_qubit_autoencoder)
    def get_input_state(p):
        return np.outer(np.conjugate(data.ground_states[p]), data.ground_states[p])
    
    for b in range(100):
        c1=[]
        for a in range(100):
            res1 = compare_state_ae(n_qubit_autoencoder,n_trash_qubit,ae)([data.ground_states[a],data.ground_states[b]])
            c1.append(res1[0])
        m1.append(c1)

    sns.heatmap(m1,yticklabels=[f'{a:.2f}' for a in range(50)],xticklabels=[f'{a:.2f}' for a in range(50)],vmin=0, vmax=1)
    plt.xlabel("h")
    plt.ylabel("h")
    plt.show()
    plot_qutip_ovelap_matrix(n_qubit_autoencoder,50)
    
def plot_qutip_ovelap_matrix(n_qubit, segm=100):
    n=[]
    for a in np.linspace(0,np.pi,segm):
        nn=[]
        for b in np.linspace(0,np.pi,segm): 
            nn.append(sp_qutip(a,n_qubit).dag()*sp_qutip(b,n_qubit))
        n.append(nn)
    n=np.array(n)
    real_part = np.real(n)
    imaginary_part = np.imag(n)
    # sns.heatmap(real_part)
    # plt.title('Real part')
    # plt.show()
    # sns.heatmap(imaginary_part)
    # plt.title('imaginary_part')
    # plt.show()
    sns.heatmap(np.real(np.abs(n)**2),xticklabels=[f'{a:.2f}' for a in range(50)],yticklabels=[f'{a:.2f}' for a in range(50)],vmin=0, vmax=1)
    plt.title('prob')
    plt.show()


In [46]:
data=get_data(iq)
data.ground_states[1]

array([-7.06962224e-01+0.j,  7.14540239e-03+0.j,  7.14540239e-03+0.j,
       -2.88585277e-04+0.j,  7.14540239e-03+0.j, -1.44322095e-04+0.j,
       -2.88585277e-04+0.j,  7.14540238e-03+0.j,  7.14540239e-03+0.j,
       -2.88585277e-04+0.j, -1.44322095e-04+0.j,  7.14540238e-03+0.j,
       -2.88585277e-04+0.j,  7.14540238e-03+0.j,  7.14540238e-03+0.j,
       -7.06962223e-01+0.j])

In [47]:
iq=4
tq=2
a=np.load('Cluster\\runs\\50\\weights0.npy')
ae= JAxutoencoder(iq,tq,None)
ae.set_weights(a)


In [48]:
compare_matrix_fidelity(iq,tq,ae)

4


ValueError: State must be a one-dimensional tensor, or two-dimensional with batching; got shape ().